In [2]:
import sqlite3
import pandas as pd
from pathlib import Path
db_loc = Path(r'C:\Main\Data\_\Database\sqlite\RespData_2204.db')
csv_loc = Path(r'C:\Users\HY_Burger\Desktop\Project')
def BasicQuery(table_in, form_type):
    if form_type == 'extube':
        table_f = 'Extube_Info'
    elif form_type == 'wean':
        table_f = 'Wean_Info'
    query_ = '''
        SELECT A.*, B.endo_end
        FROM {0} as A
        INNER JOIN {1} as B
        ON A.pid = B.PID
        GROUP by A.pid
    '''.format(table_f, table_in)
    with sqlite3.connect(db_loc) as con:
        df_sql = pd.read_sql(query_, con)
    return df_sql
remark_l = ['remark','RemarkIn','RemarkOut','RemarkInICU','RemarkOutICU']
def NotHeadRemarkPid(df):
    pid_ = []
    for i in df.index:
        row = [df.loc[i, j] for j in remark_l]
        row = [j for j in row if j]
        row = [False if '脑' in j or '神经' in j else True for j in row]
        if False in row:
            pass
        else:
            pid_.append(df.loc[i, 'pid'])
    return pid_
def InfoCollect(df):
    info = 'Total:{0}, Succ:{1}, Fail:{2}'.format(df.shape[0], len(df[df.endo_end==0]), len(df[df.endo_end==1]))
    return info

In [3]:
def MainProcess(form_name):
    df_0 = BasicQuery(form_name, form_name.split('_')[0])
    df_0.endo_end = [0 if '成功' in i else 1 for i in df_0.endo_end]
    df_1 = pd.read_csv(csv_loc/(form_name + '.csv'))
    print('Raw: \n' + InfoCollect(df_0))
    df_0 = df_0[df_0.age <= 75]
    print('Sub Old: \n' + InfoCollect(df_0))
    pid_select = NotHeadRemarkPid(df_0)
    df_0 = df_0[df_0['pid'].isin(pid_select)]
    print('Sub Head&Nerv: \n' + InfoCollect(df_0))
    df_1 = df_1[df_1['PID'].isin(list(df_0.pid))]
    pd.DataFrame.to_csv(df_1, 'prepare.csv', index=False)

In [7]:
process_mode = ['extube_sump12', 'extube_psv', 'wean_sump12', 'wean_psv']
MainProcess('wean_psv')

Raw: 
Total:855, Succ:649, Fail:206
Sub Old: 
Total:636, Succ:493, Fail:143
Sub Head&Nerv: 
Total:288, Succ:226, Fail:62
